In [1]:
import ray
import torch
from rllib_setup import get_env

In [2]:
ray.init()

2023-07-18 22:11:53,008	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.13
Ray version:,2.5.1
Dashboard:,http://127.0.0.1:8265


In [3]:
env = get_env()

In [4]:
env_name = "VJS"

In [5]:
ray.rllib.utils.check_env(env)

In [6]:
from ray.tune import register_env
register_env(env_name,lambda config: get_env())

In [7]:
from ray.rllib.policy.policy import PolicySpec
test_env = get_env()
obs_space = test_env.observation_space
act_space = test_env.action_space
def policies(agent_ids):
    return {
        str(i): PolicySpec(
            # observation_space=obs_space,
            # action_space=act_space,
            config=config.overrides(agent_id=int(i[8:])),
        )
        for i in agent_ids
    }

In [8]:
action = test_env.action_space_sample()
test_env.step(action)

({'Machine_1': array([16., 36., 16., 36., 16., 36., 16., 36., 16., 36., 16., 36., 16.,
         36., 16., 36., 16., 36., 16., 36.,  0.,  0.,  1.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.])},
 {'Machine_1': 0},
 {'Machine_1': False, '__all__': False},
 {'Machine_1': False, '__all__': False},
 {'Machine_1': {}})

In [9]:
test_env.action_space

Discrete(30)

In [10]:
from ray.tune.registry import get_trainable_cls
import os
config = (
    get_trainable_cls("MADDPG")
    .get_default_config()
    .environment(env=env_name)
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
    .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
)
(
    config.framework("tf")
    # .environment(env_config={"actions_are_logits": True})
    .training(num_steps_sampled_before_learning_starts=100)
    .multi_agent(
        policies=policies(test_env._agent_ids),
        policy_mapping_fn=lambda agent_id, episode, worker, **kwargs: str(
            agent_id
        ),
    )
)


2023-07-18 22:11:54,241	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/a3c.py` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
2023-07-18 22:11:54,243	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/a3c.py` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!


2023-07-18 22:11:54,319	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/maml/maml.py` has been deprecated. Use `rllib_contrib/maml/` instead. This will raise an error in the future!


In [11]:
%load_ext rich

In [12]:
from ray import air, tune
stop = {
    "training_iteration": 10000,
}
results = tune.Tuner(
    "MADDPG",
    run_config=air.RunConfig(stop=stop, verbose=2),
    param_space=config,
).fit()

2023-07-18 22:11:54,409	INFO algorithm_config.py:3418 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2023-07-18 22:11:54,411	INFO algorithm_config.py:3418 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.


(MADDPG pid=2060007) 2023-07-18 22:11:59,879	WARNING algorithm_config.py:643 -- Cannot create MADDPGConfig from given `config_dict`! Property __stdout_file__ not supported.
(MADDPG pid=2060007) 2023-07-18 22:11:59,880	INFO algorithm_config.py:3418 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(MADDPG pid=2060007) 2023-07-18 22:11:59,899	INFO algorithm.py:536 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=2060953) /home/yuan/ResMan/man/lib/python3.9/site-packages/gymnasium/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=2060953)   logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
(RolloutWorker pid=2060953) /home/yuan/Re

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,timers
MADDPG_VJS_0c5c8_00000,91,{},"{'num_env_steps_sampled': 100, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 91, 'num_agent_steps_trained': 0}",{},nan,{},nan,nan,nan,0,"{'learner': {}, 'num_env_steps_sampled': 100, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 91, 'num_agent_steps_trained': 0}",91,0,100,100,112.351,0,0,0,0,1,0,0,0,"{'cpu_util_percent': 2.7, 'ram_util_percent': 20.9, 'gpu_util_percent0': 0.0, 'vram_util_percent0': 0.07076009114583333, 'gpu_util_percent1': 0.0, 'vram_util_percent1': 0.0003255208333333333}",{},{},{},{},"{'episode_reward_max': nan, 'episode_reward_min': nan, 'episode_reward_mean': nan, 'episode_len_mean': nan, 'episode_media': {}, 'episodes_this_iter': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [], 'episode_lengths': []}, 'sampler_perf': {}, 'num_faulty_episodes': 0, 'connector_metrics': {}}","{'training_iteration_time_ms': 889.941, 'sample_time_ms': 880.09}"


(MADDPG pid=2060007) 2023-07-18 22:12:29,959	WARNING deprecation.py:50 -- DeprecationWarning: `SampleBatchBuilder` has been deprecated. Use `a child class of `SampleCollector`` instead. This will raise an error in the future!
2023-07-18 22:12:31,732	WARNING worker.py:2019 -- Traceback (most recent call last):
  File "python/ray/_raylet.pyx", line 1431, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 1510, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 1434, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 1438, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 1378, in ray._raylet.execute_task.function_executor
  File "/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/_private/function_manager.py", line 724, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/util/tracing/tracing_helper.py", line 464, in _resume_span
    return met